Apa itu Hash?

Hash adalah proses mengubah data menjadi nilai tetap yang unik dengan menggunakan fungsi hash. Hashing sering digunakan dalam keamanan data, seperti penyimpanan password, tanda tangan digital, dan integritas file.

In [1]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import os

# Data yang akan di-hash
data = "hello world".encode()

# Buat objek hashing menggunakan SHA-256
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())

# Masukkan data ke dalam hash function
digest.update(data)

# Dapatkan hasil hash dalam format hexdigest
hashed_output = digest.finalize().hex()

print("Hash SHA-256:", hashed_output)

Hash SHA-256: b94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9


In [2]:
# Data yang akan di-hash
data = "selamat datang di CIT".encode()

# Buat objek hashing menggunakan SHA-256
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())

# Masukkan data ke dalam hash function
digest.update(data)

# Dapatkan hasil hash dalam format hexdigest
hashed_output = digest.finalize().hex()

print("Hash SHA-256:", hashed_output)

Hash SHA-256: 37deff0f3fda2df3fca21a68a5b5f842619cea22ff0a02e28bc65994a048fe45


1. Coba run code diatas beberapa kali? Apakah hasilnya berubah
2. Ganti dengan kata yang lain apakah berubah?


jawab:
1. hasilnya tidak berubah ketika dirun beberapa kali.
2. ketika diganti dengan kata yang lain, baru hasilnya akan berubah.

In [12]:
import os, base64

# Buat salt acak (16 bytes)
salt = os.urandom(16)

# Buat objek hashing menggunakan SHA-256
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())

# Masukkan salt + data ke dalam hash function
digest.update(salt + data)

# Dapatkan hasil hash dalam format hexdigest
hashed_output_salted = digest.finalize().hex()


# Cetak hasil hashing
print("Salt (Base64):", base64.b64encode(salt).decode())
print("Hash SHA-256 dengan Salt:", hashed_output_salted)


Salt (Base64): I1lhufbWNUmoVgXSJIsfIw==
Hash SHA-256 dengan Salt: 1a3061c62b60c29064a7ca3d19ea49a0c68b8181cacc89082a0acb8489e628ed


3. Coba run code diatas beberapa kali? Apakah hasilnya berubah
4. Apa kegunaan penambahan "salt" pada kode diatas?
5. Jelaskan kegunaan metode ini untuk menangani "Rainbow Table Attack"

jawab:

3. ketika dirun berulang kali, hasilnya akan berubah terus menerus.

4. penambahan "salt" digunakan sebagai keamanan ganda (double security). jika hasilnya tidak ditambah dengan salt, maka nilai hash dari input data yang sama akan selalu menghasilkan hasil yang sama. (seperti contoh dari soal sebelumnya).

5. Rainbow Table Attack adalah metode di mana penyerang menggunakan tabel precomputed hash untuk mencocokkan nilai hash yang ada [sumber: ChatGPT]. Dengan adanya penambahan salt secara random ke hasil data yang telah melalui proses hash, tabel precomputed menjadi tidak berguna karena setiap nilai hash yang dihasilkan bersifat unik meskipun data atau pesan yang ingin disampaikan pada awalnya tetap sama.

In [126]:
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

password = "mypassword".encode()

# Buat salt acak (16 bytes)
salt = os.urandom(16)

# Gunakan PBKDF2HMAC untuk membuat hash
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
)

hashed_password = kdf.derive(password)

print("Salt (Base64):", base64.b64encode(salt).decode())
print("Derived Hash (Base64):", base64.b64encode(hashed_password).decode())


Salt (Base64): g+49HRBda8+Qf4m3JF5Trg==
Derived Hash (Base64): Pfn+9urrm58Dm+XaOaw0Zk2VfvHn4zMcBzkWGbHVkCQ=


6. Jelaskan mengapa password perlu dibuat hashnya dan bukan menggunakan enskripsi simetris atau asimetris
7. Mengapa perlu iterasi hashing yang cukup banyak pada algoritma di atas
8. Mengapa pendekatan ini berguna untuk menghadapi percobaan serangan brute force?

Jawab:

6. password perlu dibuat hashnya agar password dapat disimpan secara aman tanpa perlu menyimpan password aslinya. Sehingga jika terjadi kebocoran data, password aslinya tetap aman karena tidak bisa dikembalikan dari hash (one-way function). Dengan metode hashing, hasil hash saat proses autentikasi hanya perlu dibandingkan tanpa perlu menyimpan password aslinya.

7. iterasi hashing yang banyak diperlukan agar dapat meningkatkan keamanan data/pesan yang dikirim. Semakin banyak jumlah iterasi, maka serangan brute force atau dictionary attack dapat semakin diperlambat. Selain itu, banyaknya iterasi juga menghambat penyerangan seperti serangan GPU/ASIC karena setiap password yang ditebak membutuhkan lebih banyak daya komputasi dan waktu untuk diproses.

8. pendekatan ini sangat berguna dalam menghadapi serangan brute force karena jika terjadi kebocoran data/peretasan, maka password aslinya tetap aman karena tidak dikembalikan dari proses hashingnya. Selain itu, semakin banyak iterasi yang digunakan dan ditambahkan dalam proses hashing, semakin besar pula biaya dan waktu yang diperlukan oleh penyerang untuk membuat kombinasi password yang mungkin dan mencoba untuk meretas data.

### Buatlah code untuk enkripsi dan dekripsi sebuah pesan singkat, namun key nya adalah password yang dibuat menggunakan hash


In [5]:
import os, base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

password = "Calvin2023".encode()
salt = os.urandom(16) # Buat salt acak (16 bytes)

# Gunakan PBKDF2HMAC untuk membuat hash
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
)

key = kdf.derive(password) # key nya adalah hashed password

print(key)
print("Salt (Base64):", base64.b64encode(salt).decode())
print("Derived Hash (Base64):", base64.b64encode(key).decode())
print()

# Fungsi untuk mengenkripsi teks
def encrypt(plaintext, key):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(plaintext.encode()) + padder.finalize()

    encryptor = cipher.encryptor() 
    ciphertext = encryptor.update(padded_data) + encryptor.finalize() 

    return iv + ciphertext # Gabungkan IV dengan ciphertext

# Fungsi untuk mendekripsi teks
def decrypt(ciphertext, key):
    iv = ciphertext[:16] 
    encrypted_data = ciphertext[16:]

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    decryptor = cipher.decryptor() 
    decrypted_padded = decryptor.update(encrypted_data) + decryptor.finalize()

    unpadder = padding.PKCS7(128).unpadder() 
    plaintext = unpadder.update(decrypted_padded) + unpadder.finalize()

    return plaintext.decode()


# Proses Enkripsi dan Dekripsi:
message = "Ini adalah pesan rahasia, mohon jangan disebarluaskan!"
print(f"plain text: {message}")

# Enkripsi
ciphertext = encrypt(message, key)
print(f"Ciphertext: {ciphertext.hex()}")

# Dekripsi
decrypted_text = decrypt(ciphertext, key)
print(f"Decrypted Text: {decrypted_text}")

b'\xfc\xb0\xad\x8e\x9b\xc9\xa3\xba6Rm9\x16\x9ba\xcf\xb26b6\xbfEE\x13\xb0v-\x8fiI\xa1\xd8'
Salt (Base64): g5r59FZsxWkzu4vyprHSNQ==
Derived Hash (Base64): /LCtjpvJo7o2Um05Fpthz7I2Yja/RUUTsHYtj2lJodg=

plain text: Ini adalah pesan rahasia, mohon jangan disebarluaskan!
Ciphertext: 34928ca6dd43049f404692b8d794860c445782e3ddcd62563c87fe82ab49acb360f80bd48b7252932c2a9e1754df4f657a8ae7504a290e54cc83b48ba972a315dd463e7d3a895aae086e48ec20a6f8ad
Decrypted Text: Ini adalah pesan rahasia, mohon jangan disebarluaskan!
